## Plan of action.
We have too little data to train a model dircetly on the data. It will either be too stupid or terribly overfit.

To work with this, we will use different data, which is label with that slighly correspond to the labels we are interested in. We will train the net, then remove the last layers then and add new layers ontop. Then finetune.

Ideas:
1. don't use pooling but stride to reduce amount of parameters.
2. normailze
3. batch norm
4. dropout

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## get data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
shutil.unpack_archive("drive/MyDrive/SolarEnergyMaterials/task4_hr35z9.zip", "/content/data")
shutil.unpack_archive("data/task4_hr35z9/pretrain_features.csv.zip", "/content/data")
shutil.unpack_archive("data/task4_hr35z9/pretrain_labels.csv.zip", "/content/data")

In [16]:
import random
import numpy as np
import pandas as pd
import torch
EPSILON = 1e-10
def load_pretrain_data(batch_size = 64):
    batch_size = 64

    random.seed(17)
    test_ind = set()

    pre_train_size = 50000

    while len(test_ind) < 10000: 
        test_ind.add(random.randint(0, pre_train_size-1))

    features =[]
    labels = []

    with open("data/pretrain_features.csv", 'r') as f:
        for row in f:
            features.append(row)

    with open("data/pretrain_labels.csv", 'r') as f:
        for row in f:
            labels.append(row)

    # remove header
    features = features[1:]
    labels = labels[1:]

    # first try to note use representation of the molecules, only the extracted features
    features = [list(map(float,row.split(',')[2:])) for row in features]
    labels = [float(row.split(',')[1]) for row in labels]

    train_features = []
    train_labels = []
    test_features = []
    test_labels = []


    for i in range(len(features)):
        if i in test_ind:
            test_features.append(features[i])
            test_labels.append(labels[i])
        else:
            train_features.append(features[i])
            train_labels.append(labels[i])

    # does not seem to make sense to normalize the data since it is very sparse
    # normalize train_features
    # train_features = (train_features - np.mean(train_features, axis=0)) / (np.std(train_features, axis=0)+EPSILON)

    # normalize test_features
    # test_features = (test_features - np.mean(test_features, axis=0)) / (np.std(test_features, axis=0)+EPSILON)

    # convert into tensor dataset
    train_features = torch.tensor(train_features, dtype=torch.float)
    train_labels = torch.tensor(train_labels, dtype=torch.float)
    test_features = torch.tensor(test_features, dtype=torch.float)
    test_labels = torch.tensor(test_labels, dtype=torch.float)

    train_dataset = torch.utils.data.TensorDataset(train_features, train_labels)
    test_dataset = torch.utils.data.TensorDataset(test_features, test_labels) 
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [17]:
train_loader, test_loader = load_pretrain_data(batch_size = 64)

## train model


In [18]:
# train loop
def train_model(model, data_loader, epochs, lr=0.1, optim=None):
  if optim is None:
    optimizer = torch.optim.SGD(params=model.parameters(), lr=lr)
  else:
    optimizer = optim
  loss_fn = nn.MSELoss()
  
  model.to(device)
  for epoch in range(epochs):
    epoch_loss = 0
    for batch, (X, y) in enumerate(data_loader):
      X = X.to(device)
      y = y.to(device)
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      epoch_loss += loss.item()
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print(f"average batch loss in {epoch+1}: {epoch_loss/len(data_loader)}")
# test loop
def test_model(model, data_loader):
  loss_fn = nn.MSELoss() 
  model.to(device)
  with torch.no_grad():
    for batch, (X,y) in enumerate(data_loader):
      X = X.to(device)
      y = y.to(device)
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
    print(f"average batch loss: {loss.item()/len(data_loader)}")

In [28]:
class net(nn.Module):
    def __init__(self):
        torch.manual_seed = 17
        super(net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=6, stride=2, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=6, out_channels=16, stride=2, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=120, stride=2, kernel_size=3)
        self.fc1 = nn.Linear(120*124, 84)
        self.fc2 = nn.Linear(84, 1)

    def forward(self, x):
        x = x.unsqueeze_(dim=1)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = x.view(-1, 120*124)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.squeeze_(dim=1)
        return x
    
# using batch normalization
class normalized_net(nn.Module):
    def __init__(self):
        torch.manual_seed = 17
        super(normalized_net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=6, stride=2, kernel_size=3)
        self.bn1 = nn.BatchNorm1d(6)
        self.conv2 = nn.Conv1d(in_channels=6, out_channels=16, stride=2, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(16)
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=120, stride=2, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(120)
        self.fc1 = nn.Linear(120*124, 84)
        self.fc2 = nn.Linear(84, 1)

    def forward(self, x):
        x = x.unsqueeze_(dim=1) # need the x = x.unsqueeze_(dim=1) so gradient computation works
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.relu(self.bn3(self.conv3(x)))
        x = x.view(-1, 120*124)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.squeeze_(dim=1)    
        return x


In [20]:
test = next(iter(train_loader))[0]
dev_model = net()
out = dev_model(test)
print(out.shape)

torch.Size([64])


In [37]:
dev_model =  normalized_net()
train_model(dev_model, train_loader, epochs=15, lr=0.01)
print('---')
test_model(dev_model, train_loader)
print('---')
test_model(dev_model, test_loader)
# does not really generalize

average batch loss in 1: 0.8780323009103537
average batch loss in 2: 0.029272877728939058
average batch loss in 3: 0.026178868517279626
average batch loss in 4: 0.021420786726474763
average batch loss in 5: 0.020250629971921445
average batch loss in 6: 0.01895062441378832
average batch loss in 7: 0.018103766920417548
average batch loss in 8: 0.01771333412900567
average batch loss in 9: 0.017743518847227097
average batch loss in 10: 0.016926711418479682
average batch loss in 11: 0.014926233030110597
average batch loss in 12: 0.014913811583817006
average batch loss in 13: 0.013814114114642144
average batch loss in 14: 0.013071136762201787
average batch loss in 15: 0.01182014731541276
---
average batch loss: 1.305096596479416e-05
---
average batch loss: 0.00011510908556212286


In [36]:
dev_model =  normalized_net()
train_model(dev_model, train_loader, epochs=15, lr=0.01)
print('---')
test_model(dev_model, train_loader)
print('---')
test_model(dev_model, test_loader)
# does not really generalize

average batch loss in 1: 0.8957869546920061
average batch loss in 2: 0.021710731948912144
average batch loss in 3: 0.018483167152106763
average batch loss in 4: 0.01670166256353259
average batch loss in 5: 0.014878908500820399
average batch loss in 6: 0.013457754176855087
average batch loss in 7: 0.013481087016314269
average batch loss in 8: 0.01195252608731389
average batch loss in 9: 0.01113129082173109
average batch loss in 10: 0.011459659742936492
average batch loss in 11: 0.010420427816361188
average batch loss in 12: 0.010572349233180284
average batch loss in 13: 0.009954427929222584
average batch loss in 14: 0.009936350903660058
average batch loss in 15: 0.009829613841325045
---
average batch loss: 1.1551555991172791e-05
---
average batch loss: 6.329600408578375e-05
